In [1]:
# %pip install agents sendgrid python-dotenv

from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from openai import AsyncOpenAI



In [2]:
load_dotenv(override=True)

MODEL = 'llama-3.3-70b-versatile'
GROQ_BASE_URL = "https://api.groq.com/openai/v1"
groq_api_key = os.getenv("GROQ_API_KEY")  # Add your Groq API key to your .env file

# Point OpenAI client to Groq endpoint
groq_client = AsyncOpenAI(
    api_key=groq_api_key,
    base_url=GROQ_BASE_URL
)

groq_model = OpenAIChatCompletionsModel(model=MODEL, openai_client=groq_client)


def createAgent(name,instructions, tools = [], handoffs = [], handoff_description = "If you are unable to help, handoff to another agent."):
    agent = Agent(
        name=name,
        instructions=instructions,
        tools=tools,
        handoffs=handoffs,
        handoff_description=handoff_description,
        model=groq_model,
    )
    return agent

## Step 1: Agent workflow

In [26]:
mathTeacherInstructions = "You are a maths teacher, \
You job is to help students with their maths homework. \
You are patient and explain concepts clearly. \
You provide step by step solutions to problems."

translatorInstructions = "You are a translator, \
Translate the given text to the specified language. \
You are fluent in multiple languages and understand cultural nuances."

controllerInstructions = "You are a controller agent, \
You decide which agent to call based on the user's request. \
You can call the maths agent for math related questions and the translator agent for translation tasks. \
If the request does not match any, politely say: \
I'm not the right agent for this request."


In [42]:
maths_agent = createAgent("Professional Maths Teacher",mathTeacherInstructions)
translator_agent = createAgent("Translator",translatorInstructions)

In [43]:
maths_tools = maths_agent.as_tool(tool_name="Maths Tool", tool_description="Helps with math problems and explanations.")
translator_tools = translator_agent.as_tool(tool_name="Translator Tool", tool_description="Translates text to specified languages.")

In [ ]:
controller_agent = createAgent("Controller Agent", controllerInstructions, 
                                tools = [],
                                handoffs=[maths_agent, translator_agent])

In [49]:
result = await Runner.run(controller_agent, "Can you help me solve 2x + 3 = 7")
print(result.last_agent.name)
print(result.final_output)
result = await Runner.run(controller_agent, "i need a car, let's discuss")
print(result.last_agent.name)
print(result.final_output)
result = await Runner.run(controller_agent, "Can you translate 'Hello, how are you?' to hindi?")
print(result.last_agent.name)
print(result.final_output)

Professional Maths Teacher
I hope this explanation helped you understand how to solve the equation. Do you have any questions or would you like to practice with more examples?
Controller Agent
I'm not the right agent for this request.


BadRequestError: Error code: 400 - {'error': {'message': "tool call validation failed: attempted to call tool 'Translator Tool_args' which was not in request.tools", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '<function=Translator Tool_args>{"input": "Hello, how are you?"}</function>'}}